# Setup Requirements

## Import Libraries

In [ ]:
import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

## Specify Paths

In [ ]:
dataset_path = 'data/data.csv'
model_save_path = 'model/model.hdf5'
tflite_save_path = 'model/model.tflite'

## Setup Attributes

In [ ]:
DIMENSION = 2

NUM_CLASSES = 25

# Preprocess Data

## Load Data

In [ ]:
X_dataset = np.loadtxt(dataset_path, delimiter=',', dtype='float32', usecols=list(range(1, (21 * DIMENSION) + 1)))

In [ ]:
y_dataset = np.loadtxt(dataset_path, delimiter=',', dtype='int32', usecols=(0))

## Split Train/Test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.7, random_state=RANDOM_SEED)

# Build Model

## Construct Model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * DIMENSION, )),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

## Compile Model

In [ ]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


## Setup Callbacks

In [ ]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=80, verbose=1)

## Summary

In [ ]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

# Train Model

In [ ]:
model.fit(
    X_train,
    y_train,
    epochs=600,
    batch_size=16,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

# Evaluate Model

## Plot Confusion Matrix

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
    ax.set_ylim(len(set(y_true)), 0)
    plt.show()
    
    if report:
        print('Classification Report')
        print(classification_report(y_test, y_pred))

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, y_pred)


## Accuracy and Loss

In [ ]:
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=16)

## Predict Unseen Data

In [ ]:
predict_result = model.predict(np.array([X_test[0]]))
print(predict_result)
print(np.squeeze(predict_result))
print(np.sum((predict_result)))
print(np.argmax(np.squeeze(predict_result)))

# Convert TFLite

## Save Model

In [ ]:
model.save(model_save_path, include_optimizer=False)

## Load Model

In [ ]:
model = tf.keras.models.load_model(model_save_path)

## Quantization & Save

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)


# Inference test

## Setup Interpreter Interface

In [ ]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

## Get Tensor Details

In [ ]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

## Implement Inference

### Setup Input Tensor

In [ ]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

### Invoke Interpreter

In [ ]:
%%time
interpreter.invoke()

### Get Tensor Output

In [ ]:
tflite_results = interpreter.get_tensor(output_details[0]['index'])
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))